In [14]:
import pathlib
from docx2python import docx2python
import pdfplumber
import re
import json

### testing docx extraction

In [3]:
doc_result = docx2python('/home/kamil-solski/Documents/Python/Projekty_py/ALMA/Test_docs/Test_ALMA.docx')

text_with_placeholders = doc_result.text
print(text_with_placeholders)

Oryginalny skrypt wykorzystuje metodę A3C z biblioteki deepchem. W implementacji tej brakuje zwracanych metryk oraz progresu trenowania. W uczeniu przez wzmocnienie, skuteczna optymalizacja jest trudna bez szczegółowej analizy wydajności w miarę progresu uczenia. Aby to osiągnąć, monitoruje się szereg metryk, z których każda zapewnia wgląd w różne aspekty zachowania agenta. Metryki zaimplementowane w moich skryptach, obejmują kilka kluczowych wskaźników procesu uczenia się: skumulowane nagrody, średnie nagrody, zdyskontowane nagrody, entropię i długości epizodów. Wskaźniki te, często analizowane w wygładzonej (ang. Smoothed) formie, zapewniają kompleksowy obraz wydajności agenta, kompromisów między eksploracją a eksploatacją oraz wydajności treningu.

Wygładzanie (ang. Smoothing) wykorzystuje średnią ruchomą, czyli każdy punkt danych jest zastępowany średnią sąsiednich punktów. Ilość punktów liczonych do tych średnich jest wyznaczana przez tzw. okno. Technika ta redukuje szumy i krótko

### first version - wrong json

In [23]:
# first version
def extract_pdf_with_placeholders(pdf_path, metadata_output_path):
    sentences = []
    metadata = {}
    sentence_index = 0

    with pdfplumber.open(pdf_path) as pdf:
        for page_num, page in enumerate(pdf.pages, start=1):
            words = page.extract_words()
            images = page.images

            # Collect elements with their positions
            elements = [(float(w['top']), w['text']) for w in words]
            elements += [(float(img['top']), '[IMAGE]') for img in images]
            elements.sort(key=lambda x: x[0])

            # Combine elements into one line and collect metadata
            for position, content in elements:
                sentences.append(content)
                metadata[sentence_index] = {
                    "page": page_num,
                    "position": position,
                    "content": content,
                    "is_image": content == '[IMAGE]'
                }
                sentence_index += 1

    # Save metadata to JSON
    with open(metadata_output_path, 'w') as meta_file:
        json.dump(metadata, meta_file, indent=4)

    return " ".join(sentences)

In [24]:
pdf_path = '/home/kamil-solski/Documents/Python/Projekty_py/ALMA/Test_docs/Test_ALMA.pdf'
metadata_output_path = '/home/kamil-solski/Documents/Python/Projekty_py/ALMA/Test_docs/Test_ALMA_metadata.json'
results = extract_pdf_with_placeholders(pdf_path, metadata_output_path)
print(results)

Oryginalny skrypt wykorzystuje metodę A3C z biblioteki deepchem. W implementacji tej brakuje zwracanych metryk oraz progresu trenowania. W uczeniu przez wzmocnienie, skuteczna optymalizacja jest trudna bez szczegółowej analizy wydajności w miarę progresu uczenia. Aby to osiągnąć, monitoruje się szereg metryk, z których każda zapewnia wgląd w różne aspekty zachowania agenta. Metryki zaimplementowane w moich skryptach, obejmują kilka kluczowych wskaźników procesu uczenia się: skumulowane nagrody, średnie nagrody, zdyskontowane nagrody, entropię i długości epizodów. Wskaźniki te, często analizowane w wygładzonej (ang. Smoothed) formie, zapewniają kompleksowy obraz wydajności agenta, kompromisów między eksploracją a eksploatacją oraz wydajności treningu. Wygładzanie (ang. Smoothing) wykorzystuje średnią ruchomą, czyli każdy punkt danych jest zastępowany średnią sąsiednich punktów. Ilość punktów liczonych do tych średnich jest wyznaczana przez tzw. okno. Technika ta redukuje szumy i krótkot

### Second version - temporary chosen

In [ ]:
def extract_pdf_with_placeholders(pdf_path, metadata_path):
    output_text = ""
    metadata = []
    sentence_index = 0

    with pdfplumber.open(pdf_path) as pdf:
        for page_num, page in enumerate(pdf.pages, start=1):
            words = page.extract_words()
            images = page.images

            # Combine words and images by position
            elements = [(float(w['top']), w['text']) for w in words]
            elements += [(float(img['top']), '[/IMAGE]') for img in images]
            
            # Sort elements by vertical position
            elements.sort(key=lambda x: x[0])
            
            page_text = " ".join([content for _, content in elements])
            
            metadata.append({
                "page_number": page_num,
                "start_sentence_index": sentence_index
            })
            
            output_text += page_text + " "
            sentence_index += page_text.count('.') + page_text.count('!') + page_text.count('?')
    
    # Save metadata to JSON file
    with open(metadata_path, 'w') as json_file:
        json.dump(metadata, json_file, indent=4)
    
    return output_text.strip()

In [28]:
pdf_path = '/home/kamil-solski/Documents/Python/Projekty_py/ALMA/Test_docs/Test_ALMA.pdf'
metadata_output_path = '/home/kamil-solski/Documents/Python/Projekty_py/ALMA/Test_docs/Test_ALMA_metadata.json'
results = extract_pdf_with_placeholders(pdf_path, metadata_output_path)
print(results)

Oryginalny skrypt wykorzystuje metodę A3C z biblioteki deepchem. W implementacji tej brakuje zwracanych metryk oraz progresu trenowania. W uczeniu przez wzmocnienie, skuteczna optymalizacja jest trudna bez szczegółowej analizy wydajności w miarę progresu uczenia. Aby to osiągnąć, monitoruje się szereg metryk, z których każda zapewnia wgląd w różne aspekty zachowania agenta. Metryki zaimplementowane w moich skryptach, obejmują kilka kluczowych wskaźników procesu uczenia się: skumulowane nagrody, średnie nagrody, zdyskontowane nagrody, entropię i długości epizodów. Wskaźniki te, często analizowane w wygładzonej (ang. Smoothed) formie, zapewniają kompleksowy obraz wydajności agenta, kompromisów między eksploracją a eksploatacją oraz wydajności treningu. Wygładzanie (ang. Smoothing) wykorzystuje średnią ruchomą, czyli każdy punkt danych jest zastępowany średnią sąsiednich punktów. Ilość punktów liczonych do tych średnich jest wyznaczana przez tzw. okno. Technika ta redukuje szumy i krótkot

### Third version - included "\n"

In [29]:
def extract_pdf_with_placeholders(pdf_path, metadata_path):
    output_text = ""
    metadata = []
    sentence_index = 0

    with pdfplumber.open(pdf_path) as pdf:
        for page_num, page in enumerate(pdf.pages, start=1):
            words = page.extract_words(layout=True)
            images = page.images

            # Combine words and images by position
            elements = [(float(w['top']), w['text'] + ("\n" if w['doctop'] != words[i-1]['doctop'] else "")) for i, w in enumerate(words)]
            elements += [(float(img['top']), '[IMAGE]\n') for img in images]
            
            # Sort elements by vertical position
            elements.sort(key=lambda x: x[0])
            
            page_text = "".join([content for _, content in elements])
            
            metadata.append({
                "page_number": page_num,
                "start_sentence_index": sentence_index
            })
            
            output_text += page_text
            sentence_index += page_text.count('.') + page_text.count('!') + page_text.count('?')
    
    # Save metadata to JSON file
    with open(metadata_path, 'w') as json_file:
        json.dump(metadata, json_file, indent=4)
    
    return output_text.strip()

In [30]:
pdf_path = '/home/kamil-solski/Documents/Python/Projekty_py/ALMA/Test_docs/Test_ALMA.pdf'
metadata_output_path = '/home/kamil-solski/Documents/Python/Projekty_py/ALMA/Test_docs/Test_ALMA_metadata.json'
results = extract_pdf_with_placeholders(pdf_path, metadata_output_path)
print(results)

TypeError: WordExtractor.__init__() got an unexpected keyword argument 'layout'

### Fourth version - with layout
TODO:
- fix IMAGE position because it is wrong.

In [ ]:
def extract_pdf_with_placeholders(pdf_path):
    output = []
    with pdfplumber.open(pdf_path) as pdf:
        for page in pdf.pages:
            text = page.extract_text(layout=True) or ""
            images = page.images
            
            # Insert image placeholders inline
            for img in images:
                placeholder = '[IMAGE]'
                text += f"\n{placeholder}\n"
            
            output.append(text)
    
    return "".join(output)


In [ ]:
pdf_path = '/home/kamil-solski/Documents/Python/Projekty_py/ALMA/Test_docs/Test_ALMA.pdf'
results = extract_pdf_with_placeholders(pdf_path)
print(results)

                                                                                  
                                                                                  
                                                                                  
                                                                                  
                                                                                  
             Oryginalny skrypt wykorzystuje metodę A3C z biblioteki deepchem. W   
                                                                                  
        implementacji tej brakuje zwracanych metryk oraz progresu trenowania. W   
                                                                                  
                                                                                  
        uczeniu przez wzmocnienie, skuteczna optymalizacja jest trudna bez        
                                                                                  
    

In [ ]:
# TODO:
# - if we want to deal with images we could include layout and images at the end of creating new pdf with translation. We could use translation from chatbot, original pdf and metadata to inlude layout and images sentence wise.